# Data Link
The following section is just for Google Drive mount, in real experiment, please prepare the dataset (you can download from https://github.com/logpai/loghub)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

After running the cell above and authorizing Google Drive, run the following cell to navigate to your workspace folder named `6953final`.

In [ ]:
import os

workspace_path = '/content/drive/My Drive/6953final'

if os.path.exists(workspace_path):
    os.chdir(workspace_path)
    print(f'Successfully changed working directory to: {os.getcwd()}')
    print('Contents of the folder:')
    print(os.listdir('.'))
else:
    print(f"Error: The folder '{workspace_path}' was not found. Please ensure it exists in your Google Drive.")

# Preprocess the data

This section loads the preprocessed HDFS log dataset and converts the raw string fields into structured Python lists.
The original CSV stores the event sequence and inter-event time intervals as string representations such as "[E5,E22,E5,...]" or "[0.0, 1.0, ...]". These strings must be parsed into actual lists before they can be used to train a sequential model.

We define two helper functions:


*   parse_events extracts event template IDs from a comma-separated string and returns a list like ["E5", "E22", "E5", ...].
*   parse_timeinterval converts the time-interval field into a list of floats representing the delay between consecutive events.

After parsing, the notebook adds two new dataframe columns:


*   events – cleaned list of event IDs
*   dts_raw – raw inter-event time values (as floats)

This ensures that all downstream processing operates on consistent Python data structures rather than string literals.




In [ ]:

import pandas as pd
import numpy as np
from itertools import chain

csv_path = os.path.join("HDFS_v1", "preprocessed", "Event_traces.csv")
df = pd.read_csv(csv_path)

print("All Records:", len(df))
print(df.head(2))


In [ ]:
def parse_events(s: str):
    """
    '[E5,E22,E5,...]' -> ['E5', 'E22', 'E5', ...]
    """
    if pd.isna(s):
        return []
    s = s.strip()
    if s.startswith('[') and s.endswith(']'):
        s = s[1:-1]
    if not s:
        return []
    tokens = [tok.strip() for tok in s.split(',') if tok.strip()]
    return tokens


In [ ]:
def parse_timeinterval(s: str):
    """
    '[0.0, 1.0, 0.0, ...]' -> [0.0, 1.0, 0.0, ...]
    """
    if pd.isna(s):
        return []
    tokens = parse_events(s)
    return [float(t) for t in tokens]


In [ ]:
df["events"] = df["Features"].apply(parse_events)
df["dts_raw"] = df["TimeInterval"].apply(parse_timeinterval)



This section ensures that each sequence’s event list and its time-interval list have matching lengths.
In the HDFS dataset, a time-interval sequence typically contains one fewer element than the event sequence (because the first event has no preceding timestamp). To fix this, we define:

pad_dts_for_seq(events, dts)

*   If both lists already match in length, return the time intervals unchanged.
*   If the interval list is one item shorter, prepend a 0.0 to represent the first event.
*   Otherwise, raise an error indicating misalignment.




In [ ]:
def pad_dts_for_seq(events, dts):
    if len(dts) == len(events):
        return dts
    elif len(dts) == len(events) - 1:
        return [0.0] + dts
    else:
        raise ValueError(f"len(events)={len(events)}, len(dts)={len(dts)}")

df["dts"] = [
    pad_dts_for_seq(ev, dt)
    for ev, dt in zip(df["events"], df["dts_raw"])
]

lens = [(len(e), len(d)) for e, d in zip(df["events"], df["dts"])]
print("len(events), len(dts):", lens[:5])
assert all(le == ld for le, ld in lens), "Unequal length"


In [ ]:
PAD_ID = 0

all_events = [f"E{i}" for i in range(1, 30)]  # E1..E29
print("Event types:", len(all_events), all_events)

event2id = {e: i + 1 for i, e in enumerate(all_events)}
id2event = {i: e for e, i in event2id.items()}


In [ ]:
def encode_event_seq(seq):
    return [event2id[e] for e in seq]
df["event_ids"] = df["events"].apply(encode_event_seq)
print("original events:", df.loc[0, "events"][:15])
print("original event_ids :", df.loc[0, "event_ids"][:15])


# Event Encoding and Temporal Bucketization

After parsing the raw event sequences and time-interval lists, we convert both modalities into model ready numeric formats.
First, each event ID is mapped into a contiguous integer index using a vocabulary dictionary. This produces event_ids, a sequence of integers representing the original log templates in a form suitable for embedding layers.

Next, we analyze all non-zero time intervals across the dataset to construct a robust quantile-based bucketing scheme. Since log time intervals are typically heavy-tailed, we compute a range of quantiles (e.g., 40%, 70%, 85%, 93%, 97%, 99%, etc.) and use them as boundaries for discretization. Each interval is then clipped to a maximum threshold (HIGH_CLIP) and assigned to a bucket using np.digitize. This transforms the continuous delays into a small set of learned temporal classes that capture coarse-grained execution timing patterns without being overly sensitive to extreme outliers.

The resulting time_buckets field contains, for each sequence, a list of bucket indices aligned with event positions. A summary of bucket frequencies is printed to confirm balanced utilization across the dataset. These encoded representations—event IDs and temporal buckets—serve as the joint input features for the Transformer model.

In [ ]:
import numpy as np
from itertools import chain

all_dts = np.array(list(chain.from_iterable(df["dts"])))
nonzero_dts = all_dts[all_dts > 0]

print("25%,50%,75%,90%,99%:",
      np.quantile(nonzero_dts, [0.25, 0.5, 0.75, 0.9, 0.99]))

quantile_levels = [0.0, 0.4, 0.7, 0.85, 0.93, 0.97, 0.99, 0.995]
quantiles = np.quantile(nonzero_dts, quantile_levels)
quantiles = np.unique(quantiles)
print("quantile :", quantiles)

HIGH_CLIP = quantiles[-1]

def bucketize_dt(dt: float) -> int:
    """
    dt = 0      -> bucket 0
    0 < dt < q0 -> bucket 1
    q0 < dt < q1-> bucket 2
    ...
    dt > HIGH_CLIP -> bucket n
    """
    if dt <= 0.0:
        return 0
    dt = min(dt, HIGH_CLIP)
    idx = np.digitize(dt, quantiles, right=True)
    return int(1 + idx)

MAX_BUCKET_ID = 1 + len(quantiles)
NUM_BUCKETS   = MAX_BUCKET_ID + 1
print("time bucket all num:", NUM_BUCKETS)


In [ ]:
df["time_buckets"] = df["dts"].apply(
    lambda seq: [bucketize_dt(dt) for dt in seq]
)

all_buckets = np.array(list(chain.from_iterable(df["time_buckets"])))
unique, counts = np.unique(all_buckets, return_counts=True)
for b, c in zip(unique, counts):
    print(f"bucket {b}: {c} ({c / len(all_buckets):.3%})")


We need to determine the max sequence length by selecting the 99 percentile of all sequence length

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

seq_lens = df["events"].map(len)

print(seq_lens.describe())
print("90%,95%,99%:",
      np.quantile(seq_lens, [0.9, 0.95, 0.99]))

plt.hist(seq_lens, bins=50)
plt.xlabel("sequence length")
plt.ylabel("count")
plt.title("Event sequence length distribution")
plt.show()


In [ ]:
MAX_SEQ_LEN = int(np.quantile(seq_lens, 0.99))
print("MAX_SEQ_LEN:", MAX_SEQ_LEN)

# Dataset Encoding + PyTorch Dataset Construction
This section defines the utilities needed to convert raw event and time-bucket sequences into fixed-length training examples.
pad_or_truncate ensures that every sequence fits the model’s maximum length by trimming long traces and padding short ones with special tokens (PAD or IGNORE).
The HDFSDataset class then transforms each row into a causal-language-modeling sample:

* input_ids contain all event IDs except the last one (prefix).
* labels contain the shifted event IDs except the first one (next-token targets).
* time_ids align with the prefix to provide temporal features.
* attention_mask marks which positions are real tokens versus padding.
This dataset structure enables efficient batching and allows the Transformer to compute next-event prediction loss in a causal manner compatible with online inference.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from math import floor

IGNORE_INDEX = -100

def pad_or_truncate(seq, max_len, pad_value):
    """
    seq: list[int]
    """
    if len(seq) >= max_len:
        return seq[:max_len]
    return seq + [pad_value] * (max_len - len(seq))

class HDFSDataset(Dataset):
    def __init__(self, df, max_len):
        self.df = df
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        events = row["event_ids"]
        tb     = row["time_buckets"]

        input_tokens  = events[:-1]
        target_tokens = events[1:]
        time_tokens   = tb[:-1]

        effective_len = len(input_tokens)

        input_ids = pad_or_truncate(input_tokens,  self.max_len, PAD_ID)
        time_ids  = pad_or_truncate(time_tokens,   self.max_len, 0)
        labels    = pad_or_truncate(target_tokens, self.max_len, IGNORE_INDEX)

        attn_mask = [1] * min(effective_len, self.max_len) + \
                    [0] * max(0, self.max_len - effective_len)

        return {
            "input_ids":     torch.tensor(input_ids, dtype=torch.long),
            "time_ids":      torch.tensor(time_ids,  dtype=torch.long),
            "attention_mask":torch.tensor(attn_mask, dtype=torch.long),
            "labels":        torch.tensor(labels,    dtype=torch.long),
        }


# Train/Validation Split and DataLoader Construction

This part creates the train/validation split used for model training.
Only normal (Success) traces are used, since the model is trained in a self-supervised next-event prediction setting. The dataset is randomly divided into training and validation subsets using an 70/30 split.

Afterward, PyTorch DataLoaders are constructed for both partitions, enabling minibatch training with shuffling, multiprocessing data loading, and pinned memory for faster GPU transfer.
This setup provides an efficient input pipeline delivering batches of padded event IDs, time-bucket IDs, attention masks, and labels to the model.

In [ ]:
from sklearn.model_selection import train_test_split

# small_df = df.sample(n = 50000, random_state=42)
success_df = df[df["Label"] == "Success"].reset_index(drop=True)
fail_df = df[df["Label"] == "Fail"].reset_index(drop=True)

train_df, val_df = train_test_split(
    success_df,
    test_size=0.3,
    random_state=42,
    shuffle=True
)

train_dataset = HDFSDataset(train_df, max_len=MAX_SEQ_LEN)
val_dataset   = HDFSDataset(val_df,   max_len=MAX_SEQ_LEN)

BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=2, pin_memory=True)

batch = next(iter(train_loader))
for k, v in batch.items():
    print(k, v.shape)


# Transformer Model Skeleton
This section defines the core components of the causal Transformer used for next-event prediction in log sequences.
The embedding module maps three input channels—event IDs, positional indices, and time-bucket IDs—into a shared hidden space. The HDFSCausalTransformer stacks multiple masked self-attention layers, ensuring each position attends only to its historical context while ignoring padded tokens through a key-padding mask. The model outputs token-level logits through a final linear layer. A dedicated loss function flattens predictions and labels, applies cross-entropy with IGNORE_INDEX masking, and computes token prediction accuracy.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

VOCAB_SIZE = len(all_events) + 1
NUM_TIME_BUCKETS = NUM_BUCKETS

print("VOCAB_SIZE:", VOCAB_SIZE)
print("NUM_TIME_BUCKETS:", NUM_TIME_BUCKETS)
print("MAX_SEQ_LEN:", MAX_SEQ_LEN)

In [ ]:
IGONORE_INDEX = -100

In [ ]:
class HDFSLogEmbedding(nn.Module):
    def __init__(self, vocab_size, num_time_buckets, max_seq_len, d_model=128, dropout=0.1, pad_id=0):
        super().__init__()
        self.token_embed = nn.Embedding(vocab_size, d_model, padding_idx=pad_id)
        self.pos_embed = nn.Embedding(max_seq_len, d_model)
        self.time_embed = nn.Embedding(num_time_buckets, d_model)

        self.dropout = nn.Dropout(dropout)

        nn.init.normal_(self.token_embed.weight, mean=0.0, std=0.02)
        nn.init.normal_(self.pos_embed.weight, mean=0.0, std=0.02)
        nn.init.normal_(self.time_embed.weight, mean=0.0, std=0.02)

    def forward(self, input_ids, time_ids):
        """
        input_ids: [batch_size, seq_len]
        time_ids:  [batch_size, seq_len]
        """
        B, T = input_ids.shape
        device = input_ids.device

        pos_ids = torch.arange(T, device=device).unsqueeze(0).expand(B, T)
        x = (
            self.token_embed(input_ids) +
            self.pos_embed(pos_ids) +
            self.time_embed(time_ids)
        )
        return self.dropout(x)

In [ ]:
class HDFSCausalTransformer(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        num_time_buckets: int,
        max_seq_len: int,
        d_model: int = 128,
        n_heads: int = 4,
        n_layers: int = 3,
        d_ff: int = 512,
        dropout: float = 0.1,
        pad_id: int = 0,
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.num_time_buckets = num_time_buckets
        self.max_seq_len = max_seq_len
        self.pad_id = pad_id
        self.d_model = d_model

        # Three type of embedding, will be mixed
        self.token_embed = nn.Embedding(vocab_size, d_model, padding_idx=pad_id)
        self.pos_embed   = nn.Embedding(max_seq_len, d_model)
        self.time_embed  = nn.Embedding(num_time_buckets, d_model)

        self.embed_dropout = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_ff,
            dropout=dropout,
            batch_first=True,
            activation="gelu",
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)

        causal_mask = torch.triu(
            torch.ones(max_seq_len, max_seq_len, dtype=torch.bool),
            diagonal=1
        )
        self.register_buffer("causal_mask", causal_mask, persistent=False)

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.token_embed.weight, mean=0.0, std=0.02)
        nn.init.normal_(self.pos_embed.weight,   mean=0.0, std=0.02)
        nn.init.normal_(self.time_embed.weight,  mean=0.0, std=0.02)
        nn.init.normal_(self.lm_head.weight,     mean=0.0, std=0.02)

    def forward(
        self,
        input_ids: torch.Tensor,
        time_ids: torch.Tensor,
        attention_mask: torch.Tensor,
    ):
        B, T = input_ids.shape
        device = input_ids.device

        pos_ids = torch.arange(T, device=device).unsqueeze(0).expand(B, T)

        tok_emb  = self.token_embed(input_ids)
        pos_emb  = self.pos_embed(pos_ids)
        time_emb = self.time_embed(time_ids)

        x = tok_emb + pos_emb + time_emb
        x = self.embed_dropout(x)

        causal_mask = self.causal_mask[:T, :T]

        key_padding_mask = (attention_mask == 0)

        enc_out = self.encoder(
            x,
            mask=causal_mask,
            src_key_padding_mask=key_padding_mask,
        )

        logits = self.lm_head(enc_out)
        return logits

    def compute_loss(self, batch):
        logits = self(
            batch["input_ids"],
            batch["time_ids"],
            batch["attention_mask"],
        )

        B, T, V = logits.shape
        logits_flat = logits.reshape(B * T, V)
        labels_flat = batch["labels"].reshape(B * T)

        loss = F.cross_entropy(
            logits_flat,
            labels_flat,
            ignore_index=IGNORE_INDEX,
        )

        with torch.no_grad():
            pred = logits_flat.argmax(dim=-1)
            mask = labels_flat != IGNORE_INDEX
            correct = (pred == labels_flat) & mask
            total = mask.sum().item() if mask.any() else 1
            acc = correct.sum().item() / total

        return loss, acc


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = HDFSCausalTransformer(
    vocab_size=VOCAB_SIZE,
    num_time_buckets=NUM_TIME_BUCKETS,
    max_seq_len=MAX_SEQ_LEN,
    d_model=128,
    n_heads=4,
    n_layers=3,
    d_ff=512,
    dropout=0.1,
    pad_id=PAD_ID,
).to(device)

batch = next(iter(train_loader))
batch = {k: v.to(device) for k, v in batch.items()}

loss, acc = model.compute_loss(batch)
print("sanity loss:", float(loss.item()), "acc:", acc)


In [ ]:
from torch.optim import AdamW
from tqdm import tqdm

EPOCHS = 3
LR = 1e-3
optimizer = AdamW(model.parameters(), lr=LR)

train_step_offset = 0
val_step_offset = 0

history = {
    "train_steps": [],
    "train_loss": [],
    "train_acc": [],
    "val_steps": [],
    "val_loss": [],
    "val_acc": [],
}

def run_epoch(model, dataloader, optimizer=None, history=None, phase="train", max_points=40, global_step_offset=0):
    if optimizer is None:
        model.eval()
        mode = "eval"
    else:
        model.train()
        mode = "train"

    total_loss = 0.0
    total_tokens = 0
    total_correct = 0

    num_batches = len(dataloader)
    log_interval = max(1, num_batches // max_points)

    pbar = tqdm(enumerate(dataloader), total=num_batches, desc=f"{phase}", leave=False)

    for batch_idx, batch in pbar:
        batch = {k: v.to(device) for k, v in batch.items()}
        if optimizer is not None:
            optimizer.zero_grad()
        loss, acc = model.compute_loss(batch)
        if optimizer is not None:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        labels = batch["labels"]
        num_tokens = (labels != IGNORE_INDEX).sum().item()
        total_loss += loss.item() * num_tokens
        total_tokens += num_tokens
        total_correct += acc * num_tokens

        pbar.set_postfix({
            "loss": f"{loss.item():.4f}",
            "acc": f"{acc:.4f}"
        })

        if history is not None and (batch_idx % log_interval == 0):
            step = global_step_offset + batch_idx
            if phase == "train":
                history["train_steps"].append(step)
                history["train_loss"].append(loss.item())
                history["train_acc"].append(acc)
            else:
                history["val_steps"].append(step)
                history["val_loss"].append(loss.item())
                history["val_acc"].append(acc)

    avg_loss = total_loss / max(1, total_tokens)
    avg_acc = total_correct / max(1, total_tokens)
    last_step = global_step_offset + num_batches
    return avg_loss, avg_acc, last_step

# for epoch in range(1, EPOCHS + 1):
#     train_loss, train_acc = run_epoch(model, train_loader, optimizer)
#     val_loss, val_acc = run_epoch(model, val_loader, optimizer=None)

#     print(f"[Epoch {epoch}] "
#         f"train_loss={train_loss:.4f}, train_acc={train_acc:.4f} | "
#         f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")

for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc, train_step_offset = run_epoch(
        model, train_loader, optimizer, history=history,
        phase="train", max_points=40, global_step_offset=train_step_offset
    )

    val_loss, val_acc, val_step_offset = run_epoch(
        model, val_loader, optimizer=None, history=history,
        phase="val", max_points=40, global_step_offset=val_step_offset
    )
    print(f"[Epoch {epoch}] "
          f"train_loss={train_loss:.4f}, train_acc={train_acc:.4f} | "
          f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")


# Save and load the mode
Please modify the save path for further using

In [ ]:
save_path = "/content/drive/MyDrive/6953final/hdfs_transformer.pt"

torch.save({
    "model_state": model.state_dict(),
    "vocab": id2event,
    "event2id": event2id,
    "MAX_SEQ_LEN": MAX_SEQ_LEN,
    "NUM_BUCKETS": NUM_BUCKETS
}, save_path)

print("Model saved to", save_path)


In [ ]:
checkpoint_path = "/content/drive/MyDrive/6953final/hdfs_transformer.pt"

ckpt = torch.load(checkpoint_path, map_location=device)

id2event = ckpt["vocab"]
event2id = ckpt["event2id"]
MAX_SEQ_LEN = ckpt["MAX_SEQ_LEN"]
NUM_BUCKETS = ckpt["NUM_BUCKETS"]

model = HDFSCausalTransformer(
    vocab_size=len(id2event) + 1,
    time_bucket_size=NUM_BUCKETS,
    max_seq_len=MAX_SEQ_LEN,
    d_model=128,
    n_heads=4,
    num_layers=3,
    d_ff=512,
    dropout=0.1
).to(device)

model.load_state_dict(ckpt["model_state"])

model.eval()
print("Model loaded successfully!")


# Plot the trends of Accuracy and Loss

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history["train_steps"], history["train_loss"], label="train_loss")
plt.plot(history["val_steps"],   history["val_loss"],   label="val_loss")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Loss over training steps")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history["train_steps"], history["train_acc"], label="train_acc")
plt.plot(history["val_steps"],   history["val_acc"],   label="val_acc")
plt.xlabel("Step")
plt.ylabel("Accuracy")
plt.title("Accuracy over training steps")
plt.legend()

plt.tight_layout()
plt.show()


# Baseline Analysis: Simple Predictive Heuristics
The first block of code computes how often the next event in a sequence is identical to the current one. This gives a “same-as-previous” accuracy, reflecting how frequently logs simply repeat events.

The second block builds bigram transition statistics by counting how often each event is followed by every possible next event. For each event type, it records the probability of the most frequent successor, and averages these probabilities over all event types. This represents how well a simple bigram lookup table could predict the next token.

These two measurements provide reference baselines for next-event prediction. Compared with these simple heuristics, the Transformer model achieves substantially higher accuracy, confirming that it captures richer structural patterns in the logs than basic repetition or pairwise event frequencies.

In [ ]:
correct = 0
total = 0

for seq in df["event_ids"].head(20000):
    for i in range(len(seq) - 1):
        cur = seq[i]
        nxt = seq[i+1]
        if nxt == cur:
            correct += 1
        total += 1

print("same-as-prev baseline acc:", correct / total)


In [ ]:
from collections import Counter, defaultdict

pair_count = defaultdict(Counter)

for seq in df["event_ids"].head(20000):
    for i in range(len(seq) - 1):
        cur = seq[i]
        nxt = seq[i+1]
        pair_count[cur][nxt] += 1

probs = []
for cur, ctr in pair_count.items():
    total = sum(ctr.values())
    best = ctr.most_common(1)[0][1] / total
    probs.append(best)

print("average bigram:", sum(probs) / len(probs))


# Online Scoring
This is just a milestone for recording, please do not use this version

In [ ]:
import torch
import torch.nn.functional as F

def score_sequence(
        event_ids,
        time_buckets,
        model,
        id2event,
        max_seq_len=MAX_SEQ_LEN,
        top_k=5,
):
    model.eval()
    L = len(event_ids)
    if L < 2:
        raise ValueError("sequence too short (len < 2)")

    L_eff = min(L - 1, max_seq_len)

    input_ids = event_ids[:L_eff]
    time_ids = time_buckets[:L_eff]
    next_ids = event_ids[1:L_eff + 1]

    input_ids_tensor = torch.tensor(input_ids, dtype=torch.long, device=device).unsqueeze(0)
    time_ids_tensor  = torch.tensor(time_ids,  dtype=torch.long, device=device).unsqueeze(0)
    attn_mask_tensor = torch.ones_like(input_ids_tensor, dtype=torch.long, device=device)

    with torch.no_grad():
        logits = model(input_ids_tensor, time_ids_tensor, attn_mask_tensor)  # (1, T, V)
        probs  = F.softmax(logits, dim=-1)

    probs = probs[0]
    T = probs.size(0)

    results = {
        "position": [],
        "true_event_id": [],
        "true_event_str": [],
        "prob_true": [],
        "surprisal": [],
        "rank": [],
        "in_top_k": [],
    }

    for t in range(T):
        true_id = next_ids[t]
        prob_vec = probs[t]
        prob_true = float(prob_vec[true_id].item())

        eps = 1e-12
        surprisal = -float(torch.log(torch.tensor(prob_true + eps)))

        rank = int((prob_vec > prob_true).sum().item() + 1)

        results["position"].append(t + 1)
        results["true_event_id"].append(true_id)
        results["true_event_str"].append(id2event.get(true_id, f"ID{true_id}"))
        results["prob_true"].append(prob_true)
        results["surprisal"].append(surprisal)
        results["rank"].append(rank)
        results["in_top_k"].append(rank <= top_k)

    return results

In [ ]:
# row = df.iloc[2]
row = fail_df.iloc[0]

event_ids    = row["event_ids"]
time_buckets = row["time_buckets"]

print(event_ids)
print(time_buckets)
print(len(event_ids))

res = score_sequence(
    event_ids=event_ids,
    time_buckets=time_buckets,
    model=model,
    id2event=id2event,
    max_seq_len=MAX_SEQ_LEN,
    top_k=5,
)

for i in range(31):
    print(
        f"t={res['position'][i]:2d} "
        f"true={res['true_event_str'][i]:>3s} "
        f"prob={res['prob_true'][i]:.4f} "
        f"surprisal={res['surprisal'][i]:.3f} "
        f"rank={res['rank'][i]} "
        f"in_top5={res['in_top_k'][i]}"
    )


# Sliding Window Sequence Scoring
To compute a surprisal-based anomaly score for each event, the model must be evaluated in an online fashion—only past events can be used to predict the next one. Since many log sequences exceed the model’s maximum input length, a sliding-window prefix is required to provide a bounded context at every position.

The function performs this per-step evaluation.
For each position t, it extracts the valid prefix window, feeds it into the model, and obtains the predicted probability distribution for the next event. From this distribution, it derives several key diagnostics: the probability assigned to the true next event, its surprisal (negative log-probability), its rank among all candidate events, and whether it falls within the top-k predictions.

By sliding across the entire sequence, the function produces a trajectory of surprisal scores and ranks that reflects how well each event conforms to normal execution patterns. These scores serve as the foundation for online anomaly detection and early-warning visualization.

In [ ]:
def score_sequence_sliding(event_ids, time_buckets, model, id2event, top_k=5):
    model.eval()
    L = len(event_ids)

    results = {
        "position": [],
        "true_event_id": [],
        "true_event_str": [],
        "prob_true": [],
        "surprisal": [],
        "rank": [],
        "in_top_k": [],
    }

    if L < 2:
        return results
    for t in range(1, L):
        start = max(0, t - MAX_SEQ_LEN)
        prefix_ids = event_ids[start:t]
        prefix_times = time_buckets[start:t]

        input_ids_tensor = torch.tensor(prefix_ids, dtype=torch.long, device=device).unsqueeze(0)
        time_ids_tensor  = torch.tensor(prefix_times, dtype=torch.long, device=device).unsqueeze(0)
        attn_mask_tensor = torch.ones_like(input_ids_tensor, dtype=torch.long, device=device)

        with torch.no_grad():
            logits = model(input_ids_tensor, time_ids_tensor, attn_mask_tensor)
            probs = F.softmax(logits, dim=-1)[0]

        prob_vec = probs[-1]
        true_id = event_ids[t]

        prob_true = float(prob_vec[true_id].item())
        eps = 1e-12
        surprisal = -float(torch.log(torch.tensor(prob_true + eps)))
        rank = int((prob_vec > prob_true).sum().item() + 1)

        results["position"].append(t)
        results["true_event_id"].append(true_id)
        results["true_event_str"].append(id2event.get(true_id, f"ID{true_id}"))
        results["prob_true"].append(prob_true)
        results["surprisal"].append(surprisal)
        results["rank"].append(rank)
        results["in_top_k"].append(rank <= top_k)

    return results


To turn token level surprisals into an interpretable anomaly signal, we need first aggreagtes them along the sequence
compute_prefix_scores takes the per-step surprisal trace and computes two trajectories:


*   prefix_mean – the running average surprisal up to each position, reflecting how abnormal the sequence has been on average so far
*   prefix_max – the running maximum surprisal, capturing the strongest single deviation seen so far.

plot_anomaly_curve then visualizes these two curves over event positions, producing a 1-D “anomaly timeline” for a single trace. This makes it easy to see when the model first becomes surprised, how strong the spike is, and whether anomalies are transient or sustained—information that is crucial for debugging and for reasoning about early-warning behavior.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def compute_prefix_scores(sliding_res):
    s = np.array(sliding_res["surprisal"], dtype=float)

    if s.size == 0:
        return np.array([]), np.array([])

    prefix_mean = np.cumsum(s) / np.arange(1, len(s) + 1)
    prefix_max = np.maximum.accumulate(s)

    return prefix_mean, prefix_max


def plot_anomaly_curve(sliding_res, title=""):
    prefix_mean, prefix_max = compute_prefix_scores(sliding_res)
    positions = sliding_res["position"]

    if len(positions) == 0:
        print("No points to plot for this sequence.")
        return

    plt.figure(figsize=(10,4))
    plt.plot(positions, prefix_mean, marker="o", label="prefix_mean_surprisal")
    plt.plot(positions, prefix_max,  marker="x", label="prefix_max_surprisal", alpha=0.7)
    plt.xlabel("Position (t)")
    plt.ylabel("Anomaly score")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()



In [ ]:
row = fail_df.iloc[173]

event_ids    = row["event_ids"]
time_buckets = row["time_buckets"]
res_fail = score_sequence_sliding(event_ids, time_buckets, model, id2event, top_k=5)
plot_anomaly_curve(res_fail, title="Fail example - prefix anomaly curve")


In [ ]:
def sequence_final_score(sliding_res, mode="mean"):
    prefix_mean, prefix_max = compute_prefix_scores(sliding_res)
    if mode == "mean":
        return float(prefix_mean[-1])
    elif mode == "max":
        return float(prefix_max[-1])
    else:
        raise ValueError(f"Unknown mode: {mode}")


In [ ]:
def detect_first_crossing(sliding_res, tau, use="mean"):
    """
    tau: abnormal threshold
    use: "mean" -> prefix_mean_surprisal
         "max"  -> prefix_max_surprisal
    return:
      det_pos: first exceed tau position: t，if normal return None
    """
    prefix_mean, prefix_max = compute_prefix_scores(sliding_res)
    scores = prefix_mean if use == "mean" else prefix_max

    for i, s in enumerate(scores):
        if s > tau:
            return sliding_res["position"][i]

    return None


test tau, we will calculate the real tau among larger dataset

In [ ]:
tau = 1.0

det_pos = detect_first_crossing(res_fail, tau, use="mean")
print("first detection pos:", det_pos)


We need to estimates an anomaly detection threshold by analyzing a large sammple of normal (success) sequences. A subset of up to 50,000 success traces is selected, and each is scored using the sliding-window surprisal function. For every sequence, the final mean surprisal is computed as a single anomaly score summarizing its normality.

Collecting all success scores builds an empirical distribution of “normal” surprisal values. The 95th percentile of this distribution is then chosen as the threshold τ. This data-driven quantile threshold ensures that, under normal behavior, only a small controlled fraction of sequences exceed τ, providing a principled false-positive bound without relying on labeled anomaly data.

***This block will take a lot of time, you can use the calculated value for tau below***

In [ ]:
print("success_df:", len(success_df))
print("fail_df:", len(fail_df))

N_SUCCESS_EVAL = 50000
num_succ = len(success_df)

if num_succ > N_SUCCESS_EVAL:
    eval_indices = np.random.choice(num_succ, size=N_SUCCESS_EVAL, replace=False)
else:
    eval_indices = np.arange(num_succ)

success_scores = []

for i in eval_indices:
    row = success_df.iloc[i]
    event_ids = row["event_ids"]
    time_buckets = row["time_buckets"]

    sliding_res = score_sequence_sliding(event_ids, time_buckets, model, id2event, top_k=5)

    if len(sliding_res["surprisal"]) == 0:
        continue

    score = sequence_final_score(sliding_res, mode="mean")
    success_scores.append(score)

success_scores = np.array(success_scores)

print("used success samples:", success_scores.shape[0])
print("success_scores stats: mean=%.4f, std=%.4f" %
      (success_scores.mean(), success_scores.std()))

tau = np.quantile(success_scores, 0.95)
print("Chosen tau (95% quantile):", tau)


Tau obtained from 50,000 success record

In [ ]:
tau = 0.4460372090290396

# Evaluate Set
Code below this section is for evaluate the peroformance of the model, including FPR, Accuracy, ROC, CDF

In [ ]:
def evaluate_fail_set(fail_df, model, id2event, tau, use="mean"):
    total = len(fail_df)
    detected = 0
    advances = []

    for idx in range(total):
        row = fail_df.iloc[idx]
        event_ids    = row["event_ids"]
        time_buckets = row["time_buckets"]
        L = len(event_ids)

        if L < 2:
            continue

        sliding_res = score_sequence_sliding(event_ids, time_buckets, model, id2event)

        det_pos = detect_first_crossing(sliding_res, tau, use=use)

        if det_pos is not None:
            detected += 1
            advances.append((L - det_pos) / L)

    recall = detected / total
    avg_advance = np.mean(advances) if advances else 0.0

    print(f"FAIL set: total={total}, detected={detected}")
    print(f"Recall rate: {recall:.4f}")
    print(f"Average advance ratio: {avg_advance:.4f}")

    return recall, avg_advance

In [ ]:
def evaluate_success_fpr(success_df, model, id2event, tau, sample_size=2000, use="mean"):
    N = min(len(success_df), sample_size)
    idxs = np.random.choice(len(success_df), size=N, replace=False)

    false_positives = 0
    for idx in idxs:
        row = success_df.iloc[idx]
        event_ids = row["event_ids"]
        time_buckets = row["time_buckets"]

        if len(event_ids) < 2:
            continue

        sliding_res = score_sequence_sliding(event_ids, time_buckets, model, id2event, top_k=5)

        det_pos = detect_first_crossing(sliding_res, tau, use=use)
        if det_pos is not None:
            false_positives += 1
    fpr = false_positives / N
    print(f"SUCCESS set FPR: {fpr:.4f}  ({false_positives}/{N})")

    return fpr

In [ ]:
recall, avg_adv = evaluate_fail_set(
    fail_df, model, id2event, tau, use="mean"
)

fpr = evaluate_success_fpr(
    success_df, model, id2event, tau, sample_size=2000, use="mean"
)


In [ ]:
N_SUCCESS_EVAL = 5000
N_FAIL_EVAL = 5000

num_succ = len(success_df)
num_fail = len(fail_df)

succ_indices = np.random.choice(num_succ, size=min(N_SUCCESS_EVAL, num_succ), replace=False)
fail_indices = np.random.choice(num_fail, size=min(N_FAIL_EVAL, num_fail), replace=False)

success_scores = []
fail_scores = []

print("Scoring success samples...")
for i in tqdm(succ_indices):
    row = success_df.iloc[i]
    event_ids = row["event_ids"]
    time_buckets = row["time_buckets"]

    if len(event_ids) < 2:
        continue
    sliding_res = score_sequence_sliding(event_ids, time_buckets, model, id2event)
    if len(sliding_res["surprisal"]) == 0:
        continue
    score = sequence_final_score(sliding_res, mode="mean")
    success_scores.append(score)

success_scores = np.array(success_scores)

print("Scoring fail samples...")
for i in tqdm(fail_indices):
    row = fail_df.iloc[i]
    event_ids = row["event_ids"]
    time_buckets = row["time_buckets"]

    if len(event_ids) < 2:
        continue

    sliding_res = score_sequence_sliding(event_ids, time_buckets, model, id2event)
    if len(sliding_res["surprisal"]) == 0:
        continue

    score = sequence_final_score(sliding_res, mode="mean")
    fail_scores.append(score)

fail_scores = np.array(fail_scores)
print("success_scores:", success_scores.shape, "fail_scores:", fail_scores.shape)
print("success mean/std:", success_scores.mean(), success_scores.std())
print("fail mean/std:", fail_scores.mean(), fail_scores.std())

In [ ]:
import matplotlib.pyplot as plt

def compute_roc_points(success_scores, fail_scores, num_thresholds=100):
    all_scores = np.concatenate([success_scores, fail_scores])

    qs = np.linspace(0.01, 0.99, num_thresholds)
    taus = np.quantile(all_scores, qs)

    tprs = []
    fprs = []
    for tau in taus:
        tpr = np.mean(fail_scores > tau)
        fpr = np.mean(success_scores > tau)
        tprs.append(tpr)
        fprs.append(fpr)
    return np.array(fprs), np.array(tprs), taus

fprs, tprs, taus = compute_roc_points(success_scores, fail_scores, num_thresholds=100)
plt.figure(figsize=(6,6))
plt.plot(fprs, tprs, marker="o", linewidth=1)
plt.plot([0,1], [0,1], linestyle="--")
plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("ROC-like curve (score = avg surprisal)")
plt.grid(True)
plt.tight_layout()
plt.show()

try:
    from sklearn.metrics import roc_auc_score
    y_true = np.concatenate([np.zeros_like(success_scores), np.ones_like(fail_scores)])
    y_score = np.concatenate([success_scores, fail_scores])
    auc = roc_auc_score(y_true, y_score)
    print("AUC:", auc)
except ImportError:
    print("sklearn.metrics.roc_auc_score not available")

In [ ]:
plt.figure(figsize=(8,4))
bins = 50

plt.hist(success_scores, bins=bins, alpha=0.6, label="Success", density=True)
plt.hist(fail_scores,    bins=bins, alpha=0.6, label="Fail",    density=True)

plt.xlabel("Final mean surprisal")
plt.ylabel("Density")
plt.title("Score distribution: Success vs Fail")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
def plot_cdf(data, label):
    xs = np.sort(data)
    ys = np.linspace(0, 1, len(xs), endpoint=False)
    plt.plot(xs, ys, label=label)

plt.figure(figsize=(8,4))
plot_cdf(success_scores, "Success")
plot_cdf(fail_scores, "Fail")
plt.xlabel("Final mean surprisal")
plt.ylabel("CDF")
plt.title("CDF of Scores")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
idx_succ = np.random.randint(len(success_df))
row_succ = success_df.iloc[idx_succ]
succ_events    = row_succ["event_ids"]
succ_time_buck = row_succ["time_buckets"]

sliding_succ = score_sequence_sliding(succ_events, succ_time_buck, model, id2event)
print("Random success idx:", idx_succ, "len:", len(succ_events))
plot_anomaly_curve(sliding_succ, title=f"Success example (idx={idx_succ})")


In [ ]:
idx_fail = np.random.randint(len(fail_df))
row_fail = fail_df.iloc[idx_fail]
fail_events    = row_fail["event_ids"]
fail_time_buck = row_fail["time_buckets"]

sliding_fail = score_sequence_sliding(fail_events, fail_time_buck, model, id2event)
print("Random fail idx:", idx_fail, "len:", len(fail_events))
plot_anomaly_curve(sliding_fail, title=f"Fail example (idx={idx_fail})")


In [ ]:
succ_best_idx = np.argmin(success_scores)
succ_row = success_df.iloc[succ_indices[succ_best_idx]]

sliding_succ_best = score_sequence_sliding(
    succ_row["event_ids"],
    succ_row["time_buckets"],
    model,
    id2event
)
plot_anomaly_curve(sliding_succ_best, title="Most normal success (lowest score)")

fail_worst_idx = np.argmax(fail_scores)
fail_row = fail_df.iloc[fail_indices[fail_worst_idx]]

sliding_fail_worst = score_sequence_sliding(
    fail_row["event_ids"],
    fail_row["time_buckets"],
    model,
    id2event
)
plot_anomaly_curve(sliding_fail_worst, title="Most abnormal fail (highest score)")
